In [1]:
import tensorflow as tf
from tensorflow.keras import layers,models,optimizers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [2]:
(x_train,y_train),(x_test,y_test)=mnist.load_data()
x_train=x_train.astype('float32')/255.0
x_test=x_test.astype("float32")/255.0

In [3]:
y_train=to_categorical(y_train,10)
y_test=to_categorical(y_test,10)

In [4]:
base_model=tf.keras.applications.MobileNetV2(input_shape=(244,244,3),include_top=False,weights='imagenet')

C:\Users\Ankita\AppData\Local\Temp\ipykernel_15056\892562086.py:1: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model=tf.keras.applications.MobileNetV2(input_shape=(244,244,3),include_top=False,weights='imagenet')


In [5]:
model=models.Sequential()
model.add(layers.Reshape((28,28,1),input_shape=(28,28)))
model.add(layers.Conv2D(32,(3,3),activation='relu'))
model.add(layers.MaxPool2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(256,activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10,activation='softmax'))


d:\anaconda\lib\site-packages\keras\src\layers\reshaping\reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [6]:
for layer in base_model.layers:
    layer.trainable=False

In [7]:
model.compile(optimizer=optimizers.Adam(learning_rate=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

In [12]:
epochs=5
batch_size=32
train_dataset=tf.data.Dataset.from_tensor_slices((x_train,y_train)).shuffle(60000).batch(batch_size)
val_dataset=tf.data.Dataset.from_tensor_slices((x_test,y_test)).batch(batch_size)
optimizer=optimizers.Adam(learning_rate=0.001)

In [13]:
for epoch in range(epochs):
    print(f'Epoch {epoch+1}/{epochs}')
    for images,labels in train_dataset:
        with tf.GradientTape() as tape:
            prediction=model(images)
            loss=tf.keras.losses.categorical_crossentropy(labels,prediction)
            gradient=tape.gradient(loss,model.trainable_variables)
            optimizer.apply_gradients(zip(gradient,model.trainable_variables))

    accuracy=tf.metrics.CategoricalAccuracy()
    for image,label in val_dataset:
        prediction=model(image)
        accuracy.update_state(label,prediction)
    val_acc=accuracy.result().numpy()
    print(f'Validation accuracy:{val_acc}')

Epoch 1/5
Validation accuracy:0.986299991607666
Epoch 2/5
Validation accuracy:0.9872999787330627
Epoch 3/5
Validation accuracy:0.9872000217437744
Epoch 4/5
Validation accuracy:0.9868999719619751
Epoch 5/5
Validation accuracy:0.9890000224113464


In [15]:
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {test_accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9869 - loss: 0.0524
Test Accuracy: 0.9890000224113464
